# Asynchronous Programming

All network I/O is done with coroutines in `asyncio`, but not file I/O. However, file I/O is also "blocking"——in the sense that reading/writing files takes thousands of times longer than reading/writing to RAM. If you're using Network-Attached Storage, it may even involve network I/O under te covers. 

Since Python 3.9, the `asyncio.to_thread` coroutine makes it easy tot delegate file I/O to a thread pool provided by `asyncio`. 

## Throttling Requests with a Semaphore

An `asyncio.Semaphore` has an internal counter that is decremented whenever we `await` on the `.acquire()` coroutine method, and incremented when we call the `.release()` method——which is not coroutine because it never blocks. 

Awating on `.acquire()` cause no delay when the counter is greater than zero, but if the counter is zero, `.acquire()` suspends the awaiting coroutine until some other coroutine calls `.release()` on the same `Semaphore`. 